# Agentic Patterns: ReAct, Plan-Execute, and Reflection

**Learning Objectives:**
- Understand core agentic design patterns
- Build ReAct pattern from scratch
- Use helper functions for quick agent creation
- Implement Plan-Execute and Reflection patterns
- Know when to use manual graphs vs helpers

**Prerequisites:** Notebooks 01, 02, 03 (LangGraph Basics, Tools, Agentic RAG)


---

## Section 1: Introduction to Agentic Patterns

In notebooks 01-03, you built agents from scratch using StateGraph. But there are **common patterns** that appear repeatedly in agent design.

### What Are Agentic Patterns?

Agentic patterns are reusable blueprints for building agents.
**Instead of asking:** “How do I design this agent from scratch?”
**You ask:** “Which pattern fits this problem best?”

Think of patterns like design templates for reasoning + tool usage.


### The Three Core Agentic Patterns

**ReAct** (Reason + Act)
- Think → Act → Observe → Repeat
- Most common pattern for tool-using agents
- What you've been building in previous notebooks

**Plan-Execute**
- Plan all steps upfront → Execute each step
- Better for complex multi-step tasks
- Can replan if execution fails

**Reflection**
- Generate → Self-critique → Refine → Return
- Best for quality-critical outputs
- Similar to Self-RAG but more general


### Why Learn Patterns?

1. **Faster development** - Don't reinvent the wheel
2. **Better designs** - Learn from proven approaches
3. **Common language** - Communicate with other developers
4. **Production ready** - Used in real systems

**Today you'll master all three patterns!**

---
## Section 2: Setup

In [ ]:
# Install required packages
!pip install -q langgraph langchain langchain-openai python-dotenv

In [ ]:
# Imports
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode, create_react_agent
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from IPython.display import Image, display
from typing import Literal, TypedDict, Annotated
import operator
import os

print("✅ All imports successful")

In [ ]:
# Load API key
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found!")

print("✅ API key loaded")

In [ ]:
# Initialize LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    api_key=openai_api_key
)

print(f"✅ LLM initialized: {llm.model_name}")

---
## Section 3: ReAct Pattern - The Foundation

### What is ReAct?

**ReAct = Reasoning + Acting**

The agent alternates between thinking and acting:

```
User Query
   ↓
1. REASON: "I need to calculate this"
   ↓
2. ACT: Call calculator tool
   ↓
3. OBSERVE: Look at the tool result
   ↓
4. REASON: "Now I have the answer"
   ↓
5. RESPOND: Return final answer
```

This is what you've been building in Notebooks 02-03!

### Create Sample Tools

In [ ]:
@tool
def calculator(expression: str) -> str:
    """
    Calculate mathematical expressions.
    
    Args:
        expression: Math expression like "2 + 2" or "15 * 37"
    """
    try:
        result = eval(expression, {"__builtins__": {}}, {})
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"

@tool
def search(query: str) -> str:
    """
    Search for information (simulated).
    
    Args:
        query: The search query
    """
    # Simulated search results
    knowledge = {
        "python": "Python is a high-level programming language created in 1991.",
        "langgraph": "LangGraph is a framework for building stateful multi-actor applications.",
        "react": "ReAct is an agent pattern that combines reasoning and acting."
    }
    
    for key, value in knowledge.items():
        if key in query.lower():
            return value
    
    return "No information found."

tools = [calculator, search]
print("✅ Tools created")

---
## Section 4: ReAct - Manual Implementation

First, let's build ReAct **from scratch** to understand how it works.

In [ ]:
# Bind tools to LLM
llm_with_tools = llm.bind_tools(tools)

# System prompt
react_prompt = SystemMessage(content="""You are a helpful assistant with tools.

Use calculator for math and search for information.
Think step-by-step before using tools.""")

# Define nodes
def react_assistant(state: MessagesState) -> dict:
    """Agent node - reasons and decides which tool to use."""
    messages = [react_prompt] + state["messages"]
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    """Route to tools or end."""
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        return "tools"
    return "__end__"

# Build graph
react_builder = StateGraph(MessagesState)
react_builder.add_node("assistant", react_assistant)
react_builder.add_node("tools", ToolNode(tools))

react_builder.add_edge(START, "assistant")
react_builder.add_conditional_edges(
    "assistant",
    should_continue,
    {"tools": "tools", "__end__": END}
)
react_builder.add_edge("tools", "assistant")  # Loop back for multi-step reasoning

react_agent_manual = react_builder.compile(checkpointer=MemorySaver())

print("✅ ReAct agent (manual) created")

In [ ]:
# Visualize
try:
    display(Image(react_agent_manual.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"Could not display graph: {e}")
    print("Graph: START → assistant ↔ tools → END")

**🎨 Notice the cycle:** assistant ↔ tools allows multi-step reasoning!

### Test Manual ReAct

In [ ]:
def test_agent(agent, query: str, agent_name: str = "Agent"):
    print(f"\n{'='*80}")
    print(f"👤 User: {query}")
    print(f"🤖 {agent_name} (FULL TRACE)")
    print(f"{'='*80}\n")

    result = agent.invoke(
        {"messages": [HumanMessage(content=query)]},
        config={"configurable": {"thread_id": f"test_{agent_name}"}}
    )

    for i, msg in enumerate(result["messages"]):
        print(f"\n--- Step {i+1} ---")

        if isinstance(msg, HumanMessage):
            print("👤 Human:")
            print(msg.content)

        elif isinstance(msg, AIMessage):
            print("🤖 Assistant:")
            print(msg.content)

            # 🔥 THIS IS IMPORTANT
            if msg.tool_calls:
                print("\n🛠 Tool Calls:")
                for tc in msg.tool_calls:
                    print(f"  • Tool name: {tc['name']}")
                    print(f"  • Arguments: {tc['args']}")

        elif isinstance(msg, ToolMessage):
            print("🧰 Tool Result:")
            print(f"Tool: {msg.name}")
            print(f"Output: {msg.content}")

    final_answer = result["messages"][-1].content

    print(f"\n{'='*80}")
    print("✅ FINAL ANSWER:")
    print(final_answer)
    print(f"{'='*80}\n")

    return final_answer

In [ ]:
# Multi-step test
test_agent(
    react_agent_manual,
    "What is react ?",
    "Manual ReAct_0"
)

In [ ]:
# Multi-step test
test_agent(
    react_agent_manual,
    "Search for information about Python, then calculate 2 ** 10",
    "Manual ReAct_2"
)

**🎯 Observe:** Agent uses MULTIPLE tools in sequence - this is ReAct!

In [ ]:
# Multi-step test
test_agent(
    react_agent_manual,
    "What is LLM",
    "Manual ReAct_33"
)

**🎯 Observe:** Seach couldn't find any information, the agent has to respond through the llm!

---
## Section 5: ReAct - Using Helper Functions

Now let's see the **shortcut** way using LangGraph's helper function.

### Why Helpers Exist

Building graphs manually is great for learning, but:
- **Repetitive** for common patterns
- **Time-consuming** for simple agents
- **Error-prone** if you forget edges

Helpers create the graph structure for you!

In [ ]:
# Using create_react_agent helper
react_agent_helper = create_react_agent(
    model=llm,
    tools=tools,
    prompt=react_prompt
)

print("✅ ReAct agent (helper) created")
print("\n⚠️ Note: create_react_agent may show deprecation warning.")
print("   This is OK - it's being moved to langchain.agents but still works!")

**💡 That's it!** One function call replaces all our manual graph building.

### Test Helper ReAct

In [ ]:
test_agent(react_agent_helper, "What is 123 * 456?", "Helper ReAct_0")

In [ ]:
from langchain.agents import create_agent

create_agent_helper = create_agent(
    model=llm,
    tools=tools,
    system_prompt=react_prompt
)

In [ ]:
test_agent(create_agent_helper, "What is 123 * 456?", "Helper ReAct_1")

In [ ]:
# Same multi-step test
test_agent(
    react_agent_helper,
    "Search for information about LangGraph, then calculate 15 * 25",
    "Helper ReAct_0"
)

**🎉 Same behavior!** The helper creates the same graph structure internally.

---
## Section 6: Manual vs Helper - When to Use Which?

### Code Comparison

**Manual (20+ lines):**
```python
llm_with_tools = llm.bind_tools(tools)
def assistant(state): ...
def should_continue(state): ...

builder = StateGraph(MessagesState)
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))
builder.add_edge(START, "assistant")
builder.add_conditional_edges(...)
builder.add_edge("tools", "assistant")
agent = builder.compile()
```

**Helper (3 lines):**
```python
agent = create_react_agent(
    model=llm, tools=tools, prompt=prompt
)
```

### Feature Comparison

| Feature | Manual Building | Helper Function |
|---------|----------------|----------------|
| **Lines of code** | 20-30 | 3-5 |
| **Learning value** | ✅ High | ⚠️ Low (black box) |
| **Flexibility** | ✅ Full control | ❌ Limited |
| **Speed** | ⚠️ Slow to write | ✅ Very fast |
| **Custom patterns** | ✅ Any pattern | ❌ Only ReAct |
| **Debugging** | ✅ See everything | ⚠️ Hidden internals |
| **Production** | ✅ Both work equally well | ✅ Both work equally well |

### Decision Guide

**Use Manual Building When:**
- ✅ Learning LangGraph
- ✅ Need custom patterns (Plan-Execute, Self-RAG, CRAG)
- ✅ Need fine control over graph structure
- ✅ Building multi-agent systems
- ✅ Debugging complex behavior

**Use Helper Functions When:**
- ✅ Simple ReAct pattern is enough
- ✅ Rapid prototyping
- ✅ You understand how it works internally
- ✅ Standard tool-calling agent
- ✅ Production code (if ReAct fits your needs)

### Progression Strategy

**Beginner:** Build manually to learn  
↓  
**Intermediate:** Use helpers for simple cases, manual for complex  
↓  
**Advanced:** Choose based on requirements, not convenience  